# Title

In [ ]:
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'  # always print last expr.
%config InlineBackend.figure_format = 'svg'
%load_ext autoreload
%autoreload 2
%matplotlib inline
import pandas

pandas.options.display.max_rows = 6

In [ ]:
from tsdm.datasets import BaseDataset

In [ ]:
class USHCN(BaseDataset):
    url = "https://cdiac.ess-dive.lbl.gov/ftp/ushcn_daily/"

In [ ]:
# USHCN.download()

# State Codes

In [ ]:
# Best viewed with elastic tabstops!
state_codes = """
State	Postal Abbr.	FIPS code
Alabama	AL	01
Alaska	AK	02
Arizona	AZ	04
Arkansas	AR	05
California	CA	06
Colorado	CO	08
Connecticut	CT	09
Delaware	DE	10
District of Columbia	DC	11
Florida	FL	12
Georgia	GA	13
Hawaii	HI	15
Idaho	ID	16
Illinois	IL	17
Indiana	IN	18
Iowa	IA	19
Kansas	KS	20
Kentucky	KY	21
Louisiana	LA	22
Maine	ME	23
Maryland	MD	24
Massachusetts	MA	25
Michigan	MI	26
Minnesota	MN	27
Mississippi	MS	28
Missouri	MO	29
Montana	MT	30
Nebraska	NE	31
Nevada	NV	32
New Hampshire	NH	33
New Jersey	NJ	34
New Mexico	NM	35
New York	NY	36
North Carolina	NC	37
North Dakota	ND	38
Ohio	OH	39
Oklahoma	OK	40
Oregon	OR	41
Pennsylvania	PA	42
Puerto Rico	PR	72
Rhode Island	RI	44
South Carolina	SC	45
South Dakota	SD	46
Tennessee	TN	47
Texas	TX	48
Utah	UT	49
Vermont	VT	50
Virginia	VA	51
Virgin Islands	VI	78
Washington	WA	53
West Virginia	WV	54
Wisconsin	WI	55
Wyoming	WY	56
"""

state_dtypes = {
    "State": pandas.StringDtype(),
    "Postal Abbr.": pandas.CategoricalDtype(ordered=True),
    "FIPS code": pandas.CategoricalDtype(ordered=True),
}

In [ ]:
from io import StringIO

states = pandas.read_csv(
    StringIO(state_codes), sep="\t", dtype=state_dtypes, index_col="Postal Abbr."
)

# Stations Meta-Data

In [ ]:
station_colspecs = {
    "COOP_ID": (1, 6),
    "LATITUDE": (8, 15),
    "LONGITUDE": (17, 25),
    "ELEVATION": (27, 32),
    "STATE": (34, 35),
    "NAME": (37, 66),
    "COMPONENT_1": (68, 73),
    "COMPONENT_2": (75, 80),
    "COMPONENT_3": (82, 87),
    "UTC_OFFSET": (89, 90),
}

# fix colspec to 0-index, half open interval
station_colspecs = {key: (a - 1, b) for key, (a, b) in station_colspecs.items()}

station_dtypes = {
    "COOP_ID": pandas.CategoricalDtype(ordered=True),
    "LATITUDE": pandas.Float32Dtype(),
    "LONGITUDE": pandas.Float32Dtype(),
    "ELEVATION": pandas.Float32Dtype(),
    "STATE": states.index.dtype,
    "NAME": pandas.StringDtype(),
    "COMPONENT_1": pandas.CategoricalDtype(ordered=True),
    "COMPONENT_2": pandas.CategoricalDtype(ordered=True),
    "COMPONENT_3": pandas.CategoricalDtype(ordered=True),
    "UTC_OFFSET": "timedelta64[h]",
}

station_na_values = {
    "ELEVATION": -999.9,
    "COMPONENT_1": "------",
    "COMPONENT_2": "------",
    "COMPONENT_3": "------",
}

In [ ]:
stations_filename = "ushcn-stations.txt"
stations_filepath = USHCN.rawdata_path.joinpath(stations_filename)
stations = pandas.read_fwf(
    stations_filepath,
    na_values=station_na_values,
    colspecs=list(station_colspecs.values()),
    header=0,
    names=station_colspecs,
    dtype=station_dtypes,
)
COOP_IDS = pandas.CategoricalDtype(stations.COOP_ID, ordered=True)
stations.astype(
    {
        "COOP_ID": COOP_IDS,
        "COMPONENT_1": COOP_IDS,
        "COMPONENT_2": COOP_IDS,
        "COMPONENT_3": COOP_IDS,
    }
)

# Station Data

In [ ]:
MFLAGS = pandas.CategoricalDtype(
    categories=("B", "D", "H", "K", "L", "O", "P", "T", "W")
)
QFLAGS = pandas.CategoricalDtype(
    categories=("D", "G", "I", "K", "L", "M", "N", "O", "R", "S", "T", "W", "X", "Z")
)
SFLAGS = pandas.CategoricalDtype(
    categories=(
        "0",
        "6",
        "7",
        "A",
        "B",
        "F",
        "G",
        "H",
        "K",
        "M",
        "N",
        "R",
        "S",
        "T",
        "U",
        "W",
        "X",
        "Z",
    )
)
ELEMENTS = pandas.CategoricalDtype(categories=("PRCP", "SNOW", "SNWD", "TMAX", "TMIN"))


dtypes = {
    "COOP_ID": COOP_IDS,
    "YEAR": pandas.Int16Dtype(),
    "MONTH": pandas.Int16Dtype(),
    "ELEMENT": ELEMENTS,
    "VALUE": pandas.Int16Dtype(),
    "MFLAG": MFLAGS,
    "QFLAG": QFLAGS,
    "SFLAG": SFLAGS,
}

# column start, stop, dtype
colspecs = {
    "COOP_ID": (1, 6),
    "YEAR": (7, 10),
    "MONTH": (11, 12),
    "ELEMENT": (13, 16),
}

for k, i in enumerate(range(17, 258, 8)):
    colspecs |= {
        ("VALUE", k + 1): (i, i + 4),
        ("MFLAG", k + 1): (i + 5, i + 5),
        ("QFLAG", k + 1): (i + 6, i + 6),
        ("SFLAG", k + 1): (i + 7, i + 7),
    }

    # dtype |= {
    #     f"VALUE-{k+1}" : integer,
    #     f"MFLAG-{k+1}" : mflag_types,
    #     f"QFLAG-{k+1}" : qflag_types,
    #     f"SFLAG-{k+1}" : sflag_types,
    # }


# These should coincide with the description in data_format.txt
widths = [b - a + 1 for a, b in colspecs.values()]
dtype = {
    key: (dtypes[key[0]] if isinstance(key, tuple) else dtypes[key]) for key in colspecs
}

cspec = [(a - 1, b - 1) for a, b in colspecs.values()]
# na_values = [-9999]
# ds = pandas.read_fwf("state32.txt", names=colspecs, widths=widths, header=None, dtype=dtype, na_values=-9999)

In [ ]:
from zipfile import ZipFile
import gzip

fname = "state01_AL.txt"
fpath = USHCN.rawdata_path.joinpath(fname + ".gz")

In [ ]:
%%time
with gzip.open(fpath) as file:
    ds = pandas.read_fwf(
        file, names=colspecs, widths=widths, header=None, dtype=dtype, na_values=-9999
    )

ds

# preprocessing the data

In [ ]:
id_cols = ["COOP_ID", "YEAR", "MONTH", "ELEMENT"]
data_cols = ["VALUE", "MFLAG", "QFLAG", "SFLAG"]
data_cols = [col for col in ds.columns if col not in id_cols]
columns = pandas.MultiIndex.from_tuples(ds[data_cols], names=["VAR", "DAY"])
data = pandas.DataFrame(ds[data_cols], columns=columns)
data.index.name = "INDEX"
data

In [ ]:
%%time
# Pure magic https://stackoverflow.com/a/27044843/9318372
data = data.stack(level="DAY", dropna=False).reset_index(level="DAY")

In [ ]:
%%time
data = ds[id_cols].join(data, how="inner").reset_index()
data = data.astype(dtypes | {"DAY": integer})
data = data[
    ["COOP_ID", "YEAR", "MONTH", "DAY", "ELEMENT", "MFLAG", "QFLAG", "SFLAG", "VALUE"]
]

In [ ]:
%%time
mask = pandas.isna(data[["MFLAG", "QFLAG", "SFLAG", "VALUE"]]).sum(axis=1) < 4
data = data[mask]
data = data.sort_values(by=["YEAR", "MONTH", "DAY", "COOP_ID", "ELEMENT"]).reset_index(
    drop=True
)
data

# ALternative: Use Modin for speedup

In [ ]:
rayimport os
import ray
ray.init()

os.environ["MODIN_ENGINE"] = "ray"  # Modin will use Ray
# os.environ["MODIN_ENGINE"] = "dask"  # Modin will use Dask

In [ ]:
# problem: currently only works uncompressed.

from modin import pandas as pd

fname = "us.txt"
fpath2 = USHCN.rawdata_path.joinpath(fname)

In [ ]:
%%time
ds = pd.read_fwf(
    fpath2, names=colspecs, widths=widths, header=None, na_values=-9999, dtype=dtype
)

In [ ]:
id_cols = ["COOP_ID", "YEAR", "MONTH", "ELEMENT"]
data_cols = ["VALUE", "MFLAG", "QFLAG", "SFLAG"]
data_cols = [col for col in ds.columns if col not in id_cols]
columns = pd.MultiIndex.from_tuples(ds[data_cols], names=["VAR", "DAY"])
data = pd.DataFrame(ds[data_cols], columns=columns)
# data.columns  = columns
# data.index.name="INDEX"
data

In [ ]:
pd.DataFrame(data)

In [ ]:
%%time
# Pure magic https://stackoverflow.com/a/27044843/9318372
data = data.stack(level="DAY", dropna=False).reset_index(level="DAY")

In [ ]:
%%time
data = ds[id_cols]._to_pandas().join(data, how="inner").reset_index()
data = data.astype(dtypes | {"DAY": integer})
data = data[
    ["COOP_ID", "YEAR", "MONTH", "DAY", "ELEMENT", "MFLAG", "QFLAG", "SFLAG", "VALUE"]
]

In [ ]:
%%time
mask = pandas.isna(data[["MFLAG", "QFLAG", "SFLAG", "VALUE"]]).sum(axis=1) < 4
data = data[mask]
data = data.sort_values(by=["YEAR", "MONTH", "DAY", "COOP_ID", "ELEMENT"]).reset_index(
    drop=True
)
data